# Sertif Depan

In [ ]:
# 1. Install Library (kalau belum)
# !pip install pandas fpdf openpyxl Pillow

# 2. Import Library
import pandas as pd
from fpdf import FPDF
from PIL import Image
import os

# 3. Baca Data Excel
file_path = 'data_sertifikat.xlsx'
data = pd.read_excel(file_path)

print(data.head())

# 4. Setup PDF Class
class PDF(FPDF):
    def __init__(self, orientation, unit, size):
        super().__init__(orientation, unit, size)
        # Load semua font family
        self.add_font('Poppins', '', 'fonts/Poppins-Regular.ttf', uni=True)
        self.add_font('Poppins', 'B', 'fonts/Poppins-Bold.ttf', uni=True)
        self.add_font('Poppins', 'M', 'fonts/Poppins-Medium.ttf', uni=True)
        self.add_font('Barlow', '', 'fonts/Barlow-Regular.ttf', uni=True)
        self.add_font('Barlow', 'B', 'fonts/Barlow-Bold.ttf', uni=True)

    def add_background(self, bg_path):
        self.image(bg_path, x=0, y=0, w=self.w, h=self.h)

    def add_centered_text(self, text, font_family, font_size, y_pos_mm, font_style='B', color_rgb=(3, 60, 84), max_width_ratio=0.8):
        """
        Menambahkan teks center, dengan auto resize font jika teks terlalu panjang.
        """
        self.set_font(font_family, font_style, font_size)
        self.set_text_color(*color_rgb)
        
        max_width = self.w * max_width_ratio
        text_width = self.get_string_width(text)

        # Auto-resize kalau text_width > max_width
        while text_width > max_width and font_size > 1:
            font_size -= 1
            self.set_font(font_family, font_style, font_size)
            text_width = self.get_string_width(text)

        # Posisi teks (horizontal center)
        x_pos = (self.w - text_width) / 2
        self.set_xy(x_pos, y_pos_mm)
        self.cell(text_width, 10, text)

# 5. Path Background
background_path = 'assets/background_depan.png'

# 6. Cek Ukuran Background
bg_image = Image.open(background_path)
bg_width, bg_height = bg_image.size
pdf_width = bg_width * 0.264583
pdf_height = bg_height * 0.264583

print(f"Ukuran background: {pdf_width:.2f}mm x {pdf_height:.2f}mm")

# 7. Generate Sertifikat
output_folder = 'sertifikat_depan_output'
os.makedirs(output_folder, exist_ok=True)

# Data Unik Nama + Jabatan
unique_entries = data[['Nama', 'Jabatan']].drop_duplicates()

for index, row in unique_entries.iterrows():
    nama = row['Nama']
    jabatan = row['Jabatan']

    pdf = PDF(orientation='P', unit='mm', size=(pdf_width, pdf_height))
    pdf.add_page()
    pdf.add_background(background_path)

    # Nama - auto resize jika panjang, font medium
    pdf.add_centered_text(nama, 'Poppins', 72, y_pos_mm=186, font_style='M', max_width_ratio=0.8)

    # Jabatan - auto resize jika panjang, font bold
    pdf.add_centered_text(jabatan, 'Barlow', 32, y_pos_mm=222, font_style='B', max_width_ratio=0.8)

    # Save PDF
    filename = f"{nama} - {jabatan}.pdf".replace("/", "-").replace("\\", "-")
    pdf.output(os.path.join(output_folder, filename))

print("Sertifikat halaman depan selesai dibuat di folder:", output_folder)


# Sertif Tengah

In [ ]:
# 1. Install Library (kalau belum)
# !pip install pandas fpdf openpyxl Pillow

# 2. Import Library
import pandas as pd
from fpdf import FPDF
from PIL import Image
import os

# 3. Baca Data Excel
file_path = 'data_sertifikat.xlsx'
data = pd.read_excel(file_path)

print(data.head())

# 4. Setup PDF Class
class PDF(FPDF):
    def __init__(self, orientation, unit, size):
        super().__init__(orientation, unit, size)

    def add_background(self, bg_path):
        self.image(bg_path, x=0, y=0, w=self.w, h=self.h)

# 5. Mapping Jabatan ke File Background
jabatan_background_mapping = {
    "Ketua Umum": "assets/tengah_ketua_umum.png",
    "Wakil Ketua I": "assets/tengah_wakil_ketua_1.png",
    "Wakil Ketua II": "assets/tengah_wakil_ketua_2.png",
    "Sekretaris Umum": "assets/tengah_sekretaris_umum.png",
    "Wakil Sekretaris Umum": "assets/tengah_wakil_sekretaris_umum.png",
    "Bendahara Umum": "assets/tengah_bendahara_umum.png",
    "Wakil Bendahara Umum": "assets/tengah_wakil_bendahara_umum.png",
    "Ketua Departemen": "assets/tengah_ketua_departemen.png",
    "Wakil Ketua Departemen": "assets/tengah_wakil_ketua_departemen.png",
    "Sekretaris Departemen": "assets/tengah_sekretaris_departemen.png"
}

# 6. Fungsi untuk Menentukan Jabatan Utama
def detect_jabatan(jabatan):
    for key in jabatan_background_mapping.keys():
        if jabatan.startswith(key):
            return key
    return None

# 7. Generate Sertifikat Tengah (Tanpa Text Tambahan)
output_folder = 'sertifikat_tengah_output'
os.makedirs(output_folder, exist_ok=True)

# Data Unik Nama + Jabatan
unique_entries = data[['Nama', 'Jabatan']].drop_duplicates()

for index, row in unique_entries.iterrows():
    nama = row['Nama']
    jabatan_asli = row['Jabatan']

    jabatan_utama = detect_jabatan(jabatan_asli)

    if jabatan_utama is None:
        print(f"Jabatan tidak dikenali: {jabatan_asli}. Lewatkan.")
        continue

    background_path = jabatan_background_mapping[jabatan_utama]

    if not os.path.exists(background_path):
        print(f"Background {background_path} tidak ditemukan untuk jabatan {jabatan_asli}. Lewatkan.")
        continue

    # Cek ukuran background
    bg_image = Image.open(background_path)
    bg_width, bg_height = bg_image.size
    pdf_width = bg_width * 0.264583
    pdf_height = bg_height * 0.264583

    # Generate PDF
    pdf = PDF(orientation='P', unit='mm', size=(pdf_width, pdf_height))
    pdf.add_page()
    pdf.add_background(background_path)

    # Save PDF
    filename = f"{nama} - {jabatan_asli}.pdf".replace("/", "-").replace("\\", "-")
    pdf.output(os.path.join(output_folder, filename))

print("Sertifikat tengah (tanpa teks tambahan) selesai dibuat di folder:", output_folder)


# Sertif Event

In [ ]:
# 1. Install Library Dulu (kalau belum)
# !pip install pandas fpdf openpyxl Pillow

# 2. Import Library
import pandas as pd
from fpdf import FPDF
from PIL import Image
import os

# 3. Baca data dari Excel
file_path = 'data_sertifikat.xlsx'  # Ganti dengan nama file kamu
data = pd.read_excel(file_path)

print(data.head())

# 4. Setup PDF Class
class PDF(FPDF):
    def __init__(self, orientation, unit, size):
        super().__init__(orientation, unit, size)
        self.add_font('Poppins', '', 'fonts/Poppins-Regular.ttf', uni=True)
        self.add_font('Poppins', 'B', 'fonts/Poppins-Bold.ttf', uni=True)
    
    def add_background(self, bg_path):
        self.image(bg_path, x=0, y=0, w=self.w, h=self.h)  # Full page background

    def add_table(self, acara_kepanitiaan_list, row_height=20, max_rows_per_page=10):
        col_width = self.w / 3  # Satu kolom 1/3 dari lebar halaman

        table_width = col_width * 2
        start_x = (self.w - table_width) / 2
        self.set_y(75)  # Posisi Y awal tabel, sesuaikan kalau mau

        # --- Header ---
        self.set_fill_color(0, 57, 134)  # Biru
        self.set_text_color(255, 255, 255)  # Teks putih
        self.set_x(start_x)
        self.cell(col_width, row_height, "Acara", border=1, align='C', fill=True)
        self.cell(col_width, row_height, "Kepanitiaan", border=1, align='C', fill=True)
        self.ln()

        # --- Isi Tabel ---
        self.set_text_color(0, 0, 0)  # Balik ke teks hitam
        row_count = 0  # Untuk menghitung jumlah baris
        for acara, kepanitiaan in acara_kepanitiaan_list:
            self.set_x(start_x)
            acara_text = f' {acara}'
            kepanitiaan_text = f' {kepanitiaan}'
            self.cell(col_width, row_height, acara_text, border=1, align='L')
            self.cell(col_width, row_height, kepanitiaan_text, border=1, align='L')
            self.ln()
            row_count += 1

            # Jika sudah 10 baris, tambah halaman baru
            if row_count >= max_rows_per_page:
                self.add_page()
                self.add_background(background_path)
                row_count = 0  # Reset jumlah baris

# 5. Path background sertifikat
background_path = 'assets/background_belakang.png'  # Ganti dengan file background kamu

# 6. Cek ukuran background
bg_image = Image.open(background_path)
bg_width, bg_height = bg_image.size  # dalam pixel

# Konversi ukuran pixel ke ukuran mm (1 pixel = 0.264583 mm)
pdf_width = bg_width * 0.264583
pdf_height = bg_height * 0.264583

print(f"Ukuran background: {pdf_width:.2f}mm x {pdf_height:.2f}mm")

# 7. Generate Sertifikat
output_folder = 'sertifikat_belakang_output'
os.makedirs(output_folder, exist_ok=True)

# Grouping berdasarkan Nama + Jabatan
grouped = data.groupby(['Nama', 'Jabatan'])

for (name, jabatan), group in grouped:
    acara_kepanitiaan_list = list(zip(group['Acara'], group['Kepanitiaan']))

    if not acara_kepanitiaan_list:  # Kalau kosong, skip
        continue

    pdf = PDF(orientation='P', unit='mm', size=(pdf_width, pdf_height))
    pdf.add_page()
    pdf.add_background(background_path)

    # Tabel, max row per halaman 10
    pdf.add_table(acara_kepanitiaan_list, row_height=20, max_rows_per_page=10)

    # Save PDF
    filename = f"{name} - {jabatan}.pdf".replace("/", "-").replace("\\", "-")
    pdf.output(os.path.join(output_folder, filename))


print("Sertifikat berhasil dibuat di folder:", output_folder)


# PDF Merger

In [ ]:
# 1. Install Library (kalau belum)
# !pip install pandas PyPDF2 openpyxl

# 2. Import Library
import pandas as pd
import os
from PyPDF2 import PdfMerger

# 3. Baca Data Excel
file_path = 'data_sertifikat.xlsx'
data = pd.read_excel(file_path)

print(data.head())

# 4. Path Folder Output
depan_folder = 'sertifikat_depan_output'
tengah_folder = 'sertifikat_tengah_output'
belakang_folder = 'sertifikat_belakang_output'

# 5. Output Folder Final
final_output_folder = 'sertifikat_final_output'
os.makedirs(final_output_folder, exist_ok=True)

# 6. Data Unik Nama + Jabatan
unique_entries = data[['Nama', 'Jabatan']].drop_duplicates()

# 7. Gabungkan PDF
for idx, row in enumerate(unique_entries.itertuples(), start=1):
    nama = getattr(row, 'Nama')
    jabatan = getattr(row, 'Jabatan')

    # Path semua file
    file_depan = os.path.join(depan_folder, f"{nama} - {jabatan}.pdf").replace("\\", "/")
    file_tengah = os.path.join(tengah_folder, f"{nama} - {jabatan}.pdf").replace("\\", "/")
    file_belakang = os.path.join(belakang_folder, f"{nama} - {jabatan}.pdf").replace("\\", "/")

    # List file yang akan digabung
    files_to_merge = []
    
    if os.path.exists(file_depan):
        files_to_merge.append(file_depan)
    else:
        print(f"WARNING: File depan tidak ditemukan untuk {nama} - {jabatan}.")

    if os.path.exists(file_tengah):
        files_to_merge.append(file_tengah)
    else:
        print(f"WARNING: File tengah tidak ditemukan untuk {nama} - {jabatan}.")

    if os.path.exists(file_belakang):
        files_to_merge.append(file_belakang)
    else:
        print(f"WARNING: File belakang tidak ditemukan untuk {nama} - {jabatan}.")

    # Kalau tidak ada satupun file, skip
    if not files_to_merge:
        print(f"ERROR: Tidak ada file sama sekali untuk {nama} - {jabatan}. Dilewati.")
        continue

    # Gabungkan file yang tersedia
    merger = PdfMerger()

    for file_path in files_to_merge:
        merger.append(file_path)

    filename = f"{idx:03d} {nama}.pdf".replace("/", "-").replace("\\", "-")
    output_path = os.path.join(final_output_folder, filename)

    merger.write(output_path)
    merger.close()

    print(f"Sukses gabungkan: {filename}")

print("Semua sertifikat berhasil digabungkan di folder:", final_output_folder)


print("Semua sertifikat berhasil digabungkan di folder:", final_output_folder)
